In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [ ]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '22'
DB_USER = 'grupo8'
DB_PASSWORD = 'bulla8'

In [ ]:
#Muestre todos los restaurantes que ofrezcan un plato específico (string) ingresado y que esté disponible
def consulta1(): 
    #query devuelve nombre y sucursal de los restaurantes que ofrecen el plato ingresado
    query = "SELECT Restaurant.nombre, Restaurant.sucursal \
        FROM Restaurant, Plato \
        WHERE Plato.nombre = %(nombre)s AND Plato.disponibilidad = TRUE"
    return query

#Muestre todos los pedidos de un usuario específico ingresado (email string) y su gasto mensual (solo los pedidos concretados)
def consulta2():
    query = "SELECT * \
        FROM Pedido, Usuario \
        WHERE Usuario.email = %(email)s \
        AND Pedido.usuario_id = Usuario.id" 
    
    query1 = "SELECT * \
        FROM Pedido (Pedido, Usuario \
        WHERE Usuario.email = %(email)s \
        AND Pedido.usuario_id = Usuario.id\
        AND Pedido.estado = 'entregado a cliente' \
        ) as pedidos \
                " 
        #falta el gasto mensual, tengo que pensar como hacerlo porque hay suscripciones

    return query

#Muestre todos pedidos concretados y cancelados y el valor total de ellos
def consulta3():
    query = "SELECT * \
        FROM Pedido \
        WHERE Pedido.estado IN ('entregado a cliente', 'cliente cancela', 'delivery cancela', 'restaurante cancela')"
    #falta el valor total de los pedidos
    return query

#Dado un estilo de plato ingresado por el usuario (string) , muestre todas los platos con ese tipo, los restaurantes que las ofrecen y las opciones de delivery.

def consulta4():
    query = "SELECT Plato.nombre, Res_em.nombre_rest, Res_em.sucursal, Res_em.nombre_emp \
        FROM Plato, (SELECT Restaurant.nombre, Restaurant.sucursal, Empresa.nombre \
                    FROM Restaurant, Empresa, Restaurant_empresa \
                    WHERE Restaurant.id = Restaurant_empresa.restaurant_id \
                    AND Empresa.id = Restaurant_empresa.empresa_id) as Res_em \
        WHERE Plato.estilo = %(estilo)s \
        AND Plato.restaurant_id = Res_em.id \
        AND Plato.disponibilidad = TRUE"
    return query

#Dado un estilo de plato ingresado (string) por el usuario, muestre todas las platos que pertenezcan a ese estilo y sus restricciones
def consulta5():
    query = "SELECT Restaurant.nombre, Restaturant.sucursal, Plato.nombre, Plato.restriccion \
        FROM Plato, Restaurant \
        WHERE Plato.estilo = %(estilo)s \
        AND Plato.restaurant_id = Restaurant.id \
        AND Plato.disponibilidad = TRUE"
    return query

#Dado un cliente ingresado por el usuario (email string), muestre todas las restaurantes a las que tiene acceso con sus suscripciones.

def consulta6():
    query = "SELECT Restaurant.nombre, Restaurant.sucursal, Empresa.nombre \
        FROM Restaurant, Empresa, Restaurant_empresa, Usuario, Suscripcion \
        WHERE Restaurant.id = Restaurant_empresa.restaurant_id \
        AND Empresa.id = Restaurant_empresa.empresa_id \
        AND Usuario.id = Suscripcion.usuario_id \
        AND Suscripcion.empresa_id = Empresa.id \
        AND Usuario.email = %(email)s"
    return query

# Muestre la suma de dinero gastada por cada cliente en pedidos no incluidas en planes de suscripcion



#Muestre todos los platos y los restaurantes que los ofrecen

def consulta8():
    query = "SELECT Plato.nombre, Restaurant.nombre, Restaurant.sucursal \
        FROM Plato, Restaurant \
        WHERE Plato.restaurant_id = Restaurant.id \
        AND Plato.disponibilidad = TRUE"
    return query

#Dado un numero (1-5) ingresado por el usuario, muestre todos las evaluaciones de Clientes, Delivery, Despachador superiores o iguales a  ́el


#Dado una un alergeno ingresado por el usuario (string), muestre todos los platos que lo contengan en sus ingredientes (Ejemplo: maní)
def consulta10():
    query = "SELECT Restaurant.nombre, Restaturant.sucursal, Plato.nombre \
        FROM Plato,  Restaurant \
        WHERE Plato.ingredientes LIKE %(alergeno)s \
        AND Plato.disponibilidad = TRUE \
        AND Plato.restaurant_id = Restaurant.id"
    return query

In [ ]:
def consultas(numero_consulta, **kwargs):
    conn = None
    cursor = None
    
    try:
        # Conexión con la base de datos
        conn = psycopg2.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        # Crear un cursor para ejecutar consultas
        cursor = conn.cursor()

        #consulta 1
        if numero_consulta == 1:
            try:
                query = consulta1()
                nombre_plato = kwargs.get('nombre_plato')
                cursor.execute(query, {'nombre': nombre_plato})
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 3:
            try:
                query = consulta3()
                cursor.execute(query)
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 4:
            try:
                query = consulta4()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 5:
            try:
                query = consulta5()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 6:
            try:
                query = consulta6()
                email = kwargs.get('email')
                cursor.execute(query, {'email': email})
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 8:
            try:
                query = consulta8()
                cursor.execute(query)
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 10:
            try:
                query = consulta10()
                alergeno = kwargs.get('alergeno')
                cursor.execute(query, {'alergeno': '%{}%'.format(alergeno)}) #revisar si esto funciona
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

    except (Exception, Error) as error:
        print("Error al conectarse al servidor:", error)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()